In [1]:
#import spark for python! 
import pyspark

#create the spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/10 08:11:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# imports
import pandas as pd
import numpy as np
from pydataset import data

## Create a spark data frame that contains your favorite programming languages.
* The name of the column should be language
* View the schema of the dataframe
* Output the shape of the dataframe
* Show the first 5 records in the dataframe

In [3]:
# Define a list of your favorite programming languages
favorite_languages = ["Python", "Java", "JavaScript", "C++", "Ruby"]

# Create a DataFrame with the column name "language"
df = spark.createDataFrame([(language,) for language in favorite_languages], ["language"])

# Show the DataFrame
df.show()

+----------+
|  language|
+----------+
|    Python|
|      Java|
|JavaScript|
|       C++|
|      Ruby|
+----------+



## Load the mpg dataset as a spark dataframe.

In [4]:
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [5]:
# Convert the pandas DataFrame to Spark DataFrame
df = spark.createDataFrame(mpg)

# Create a new column with the desired output message
df = df.withColumn('output', 
                   F.concat(F.lit('The '), df.year, F.lit(' '), df.manufacturer, F.lit(' '), df.model, F.lit(' has a '), df.cyl, F.lit(' cylinder engine.')))

# Transform the trans column to only contain either manual or auto
df = df.withColumn('trans', F.when(df.trans.like('%manual%'), 'manual').otherwise('auto'))

# Show the resulting DataFrame
df.show(truncate=False)

TypeError: data is already a DataFrame

In [7]:
import pyspark.sql.functions as F

In [8]:
# Load the tips dataset from the pydataset library
tips = data('tips')

# Convert the pandas DataFrame to Spark DataFrame
df = spark.createDataFrame(tips)

# Calculate the percentage of observations that are smokers
smokers_percentage = df.filter(df.smoker == 'Yes').count() / df.count() * 100

# Create a column that contains the tip percentage
df = df.withColumn('tip_percentage', F.col('tip') / F.col('total_bill') * 100)

# Calculate the average tip percentage for each combination of sex and smoker
avg_tip_percentage = df.groupby('sex', 'smoker').agg(F.avg('tip_percentage').alias('avg_tip_percentage'))

# Show the resulting DataFrame
df.show(truncate=False)
print("Percentage of observations that are smokers: {:.2f}%".format(smokers_percentage))
avg_tip_percentage.show(truncate=False)

+----------+----+------+------+---+------+----+------------------+
|total_bill|tip |sex   |smoker|day|time  |size|tip_percentage    |
+----------+----+------+------+---+------+----+------------------+
|16.99     |1.01|Female|No    |Sun|Dinner|2   |5.9446733372572105|
|10.34     |1.66|Male  |No    |Sun|Dinner|3   |16.054158607350097|
|21.01     |3.5 |Male  |No    |Sun|Dinner|3   |16.658733936220845|
|23.68     |3.31|Male  |No    |Sun|Dinner|2   |13.97804054054054 |
|24.59     |3.61|Female|No    |Sun|Dinner|4   |14.680764538430255|
|25.29     |4.71|Male  |No    |Sun|Dinner|4   |18.62396204033215 |
|8.77      |2.0 |Male  |No    |Sun|Dinner|2   |22.80501710376283 |
|26.88     |3.12|Male  |No    |Sun|Dinner|4   |11.607142857142858|
|15.04     |1.96|Male  |No    |Sun|Dinner|2   |13.031914893617023|
|14.78     |3.23|Male  |No    |Sun|Dinner|2   |21.853856562922868|
|10.27     |1.71|Male  |No    |Sun|Dinner|2   |16.65043816942551 |
|35.26     |5.0 |Female|No    |Sun|Dinner|4   |14.180374361883

+------+------+------------------+
|sex   |smoker|avg_tip_percentage|
+------+------+------------------+
|Male  |No    |16.06687151291298 |
|Female|No    |15.69209707691836 |
|Male  |Yes   |15.277117520248513|
|Female|Yes   |18.21503526994103 |
+------+------+------------------+

